In this notebook, we want to fit different models to the cleaned data from the 'data_inspection' notebook. We start with simple models like ARIMA and will work our way to pure deep learning.

### 1. VARIMA

[Medium Blog Entry](https://medium.com/mlearning-ai/multivariate-time-series-using-var-model-efa215371738)

Explanation: 

The VARIMA model combines autoregressive (AR), integrated (I), and moving average (MA) components to capture the patterns and dynamics present in the data. Let's break down each component:

- Autoregressive (AR) component: The AR component considers the relationship between each variable and its own past values. It assumes that the future values of a variable depend on its past values. The order of the AR component (AR order) determines the number of lagged observations used to predict the current value.
- Integrated (I) component: The I component involves differencing the time series data to make it stationary. Stationarity means that the statistical properties of the series, such as mean and variance, do not change over time. Differencing helps remove trends or seasonality present in the data. The order of differencing (integration order) indicates the number of times differencing is applied to achieve stationarity.
- Moving Average (MA) component: The MA component considers the relationship between each variable and the residual errors from past predictions. It assumes that the future values of a variable depend on the errors made by past predictions. The order of the MA component (MA order) represents the number of lagged forecast errors used to predict the current value.

### 2. Exponential Smoothing

### 3. Gradient Boosting Machines

### 4. Gaussian Processes

### 5. ODEs / PDEs

### 6. Prophet

[Medium Blog Entry](https://medium.com/@bobrupakroy/yes-our-favorite-fbprophet-is-back-with-multivariate-forecasting-785fbe412731)

### 6. LSTMs

[Medium Blog Entry](https://medium.com/@bobrupakroy/multi-variate-lstm-time-series-forecasting-1a736009f6d)

### 7. Analysis